# Convolutional Neural Network with Fashion MNIST

The Fashion MNIST dataset is designed to take the place of the digit MNIST dataset because many models can fairly easily solve that problem. When I ran the Convnet on MNIST after 5 epochs I was able to achieve a 97% accuracy rating. Let's see what a Convnet does on the Fashion MNIST dataset. The Logistic Regression was able to achieve 76.8% accuracy after 20 epoch and the feedforward neural network was able to achieve 87% accuracy after 20 epochs.

In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [2]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data/FashionMNIST', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)
test_dataset = dsets.MNIST(root='./data/FashionMNIST', 
                           train=False, 
                           transform=transforms.ToTensor())

In [3]:
batch_size = 100
n_iters = 12000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 7 * 7, 10) 
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out

In [5]:
model = CNNModel()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

criterion = nn.CrossEntropyLoss()

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

cuda:0


In [6]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        images = images.requires_grad_().to(device)
        labels = labels.to(device)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 1000 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                images = images.requires_grad_().to(device)
                labels = labels.to(device)
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
                    
            accuracy = 100 * correct.item() / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 1000. Loss: 0.7118532061576843. Accuracy: 77.33
Iteration: 2000. Loss: 0.43869757652282715. Accuracy: 82.17
Iteration: 3000. Loss: 0.5267837643623352. Accuracy: 82.94
Iteration: 4000. Loss: 0.5032407641410828. Accuracy: 84.77
Iteration: 5000. Loss: 0.35217875242233276. Accuracy: 85.93
Iteration: 6000. Loss: 0.45467498898506165. Accuracy: 86.19
Iteration: 7000. Loss: 0.31846532225608826. Accuracy: 86.64
Iteration: 8000. Loss: 0.46997207403182983. Accuracy: 86.3
Iteration: 9000. Loss: 0.3390321433544159. Accuracy: 87.26
Iteration: 10000. Loss: 0.3405841886997223. Accuracy: 87.84
Iteration: 11000. Loss: 0.32820990681648254. Accuracy: 85.56
Iteration: 12000. Loss: 0.2726011574268341. Accuracy: 87.79


The convnet did very similarly to the feedforward neural network obtaining almost an 88% accuracy.

So, the FMNIST dataset is much more difficult to classify than the traditional MNIST dataset, but a CNN can obtain almost 90% accuracy. 